In [1]:
#!pip install swifter --user
#!pip install numpy --user
#!pip install nltk --user
#!pip install sklearn --user
#!pip install arrow --user

In [4]:
# импорт библиотек
import json 
import numpy as np
import pandas as pd
import re
import string
import nltk
import csv
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer 
import gc
import swifter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle 
import sys
import arrow
import numpy as np
from numpy.linalg import norm
from multiprocessing import Pool
from correct import correct_numbers

J:\Anaconda3\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\siowl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\siowl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
STOP_WORDS = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+') 

In [14]:
df_business = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
df_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,921 Pearl St,"{'RestaurantsTableService': 'True', 'WiFi': 'u...",6iYb2HFDywm3zjuRg0shjw,"Gastropubs, Food, Beer Gardens, Restaurants, B...",Boulder,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1,40.017544,-105.283348,Oskar Blues Taproom,80302,86,4.0,CO
1,7000 NE Airport Way,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...",tCbdrRPZA0oiIYSmHG3J0w,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",Portland,"{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",1,45.588906,-122.593331,Flying Elephants at PDX,97218,126,4.0,OR
2,4720 Hawthorne Ave,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",bvN78flM8NLprQ1a1y5dRg,"Antiques, Fashion, Used, Vintage & Consignment...",Portland,"{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0...",1,45.511907,-122.613693,The Reclaimory,97214,13,4.5,OR
3,2566 Enterprise Rd,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...",oaepsyvc0J17qwi8cfrOWg,"Beauty & Spas, Hair Salons",Orange City,None,1,28.914482,-81.295979,Great Clips,32763,8,3.0,FL
4,1046 Memorial Dr SE,"{'GoodForKids': 'False', 'BusinessParking': '{...",PE9uqAjdw0E4-8mjGl3wVA,"Gyms, Active Life, Interval Training Gyms, Fit...",Atlanta,"{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'...",1,33.747027,-84.353424,Crossfit Terminus,30316,14,4.0,GA


In [15]:
#df_business = df_business.dropna() 
df_business = df_business[df_business['categories'].notna() & df_business['state'].notna()]
df_business_restaurants = df_business[df_business['categories'].str.contains('Restaurants') & df_business['state'].str.contains('CO')] 

del(df_business)
gc.collect()

df_business_restaurants = df_business_restaurants[df_business_restaurants['is_open'] == 1]
df_business_restaurants.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,921 Pearl St,"{'RestaurantsTableService': 'True', 'WiFi': 'u...",6iYb2HFDywm3zjuRg0shjw,"Gastropubs, Food, Beer Gardens, Restaurants, B...",Boulder,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1,40.017544,-105.283348,Oskar Blues Taproom,80302,86,4.0,CO
52,3255 28th Street,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",Of6xu3pY3eHe2yhiyz2dvg,"Restaurants, Tex-Mex, Tacos, Fast Food, Breakf...",Boulder,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,40.033678,-105.259103,Taco Bell,80301,44,1.5,CO
158,"6545 Gunpark Dr, Ste 280","{'BikeParking': 'True', 'GoodForMeal': '{'dess...",_8Hejg5Q-_izIhLvq2ocnw,"Restaurants, Mexican",Boulder,"{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",1,40.070694,-105.200376,Sancho's Authentic Mexican Restaurant,80301,220,4.0,CO
593,2124 14th St,"{'RestaurantsReservations': 'False', 'Business...",or-Kyw7kmNin1pWbu2RLJw,"Cajun/Creole, Restaurants, Breakfast & Brunch,...",Boulder,"{'Monday': '7:0-14:0', 'Tuesday': '7:0-14:0', ...",1,40.020092,-105.277655,Lucile's Creole Cafe,80302,628,4.0,CO
1481,1109 Walnut St,"{'RestaurantsReservations': 'False', 'Restaura...",Rbt9i4IDFiIBsau020X_xQ,"Nightlife, African, American (New), Restaurant...",Boulder,"{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...",1,40.017039,-105.280694,Rosetta Hall,80302,112,3.5,CO


In [16]:
print(df_business_restaurants.shape)

(497, 14)


In [20]:
keys = ["review_id", "user_id", "business_id", "stars", "text"]
with open("review_data.csv", "w",  encoding='utf-8') as fcsv:
    wr = csv.writer(fcsv)
    wr.writerow(['review_id', 'user_id', 'business_id', 'stars', 'text'])
    with open('yelp_dataset/yelp_academic_dataset_review.json', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            wr.writerow([data[keys[0]], data[keys[1]], data[keys[2]], data[keys[3]], data[keys[4]]])

In [21]:
def text_preprocess(text):
    text = tokenizer.tokenize(text.lower())
    text = [re.sub(r'[^a-z\s]', '', w) for w in text]
    text = [lemmatizer.lemmatize(w) for w in text if w not in STOP_WORDS]
    return ' '.join(text)

In [22]:
df_review = pd.read_csv("review_data.csv")
print(df_review.shape)
df_review.head()

(8635403, 5)


,review_id,user_id,business_id,stars,text
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,Apparently Prides Osteria had a rough summer a...
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,This store is pretty good. Not as great as Wal...
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,I called WVM on the recommendation of a couple...
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,I've stayed at many Marriott and Renaissance M...
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,The food is always great here. The service fro...


In [23]:
df_review = df_review[df_review['business_id'].isin(df_business_restaurants['business_id'])]
df_review["text"].fillna('', inplace=True)

In [24]:
df_review['text'] = df_review['text'].swifter.apply(text_preprocess)
df_review.shape 

(68353, 5)

In [25]:
df_review.to_csv('review_data_state.csv', index=False)

In [26]:
class MatrixFactorization(object):
    
    def __init__(self, U, I, learning_rate=0.1, lamda=0.1, num_epoches=5, num_batches=100):
        self.learning_rate = learning_rate
        self.lamda = lamda 
        self.num_epoches = num_epoches  
        self.num_batches = num_batches  
        self.U = U
        self.I = I 
        self.e = 0 

    def loss(self, ratings):
        errors = [(rat_ij - np.dot(self.U[i], self.I[j].T))**2 + self.lamda * \
              (norm(self.U[i]) + norm(self.I[j])) for i, j, rat_ij in ratings]
        return sum(errors)

    def update(self, ratings):
        for i, j, rat_ij in ratings:
            rat_ij_hat = np.dot(self.U[i], self.I[j].T)
            grad_U_i = (rat_ij_hat - rat_ij) * self.I[j] + self.lamda * self.U[i]
            grad_I_j = (rat_ij_hat - rat_ij) * self.U[i] + self.lamda * self.I[j]
            self.U[i] -= self.learning_rate * grad_U_i
            self.I[j] -= self.learning_rate * grad_I_j
    
    def fit(self, train, test):
        num_trains = train.shape[0] 
        num_tests  = test.shape[0] 
        while self.e < self.num_epoches:
            self.e += 1
            mixed_order = np.arange(num_trains)
            np.random.shuffle(mixed_order)
            avg_train_loss = [] 
            avg_test_loss  = [] 
            batch_size = int(num_trains / self.num_batches) 
            for batch in range(self.num_batches):
                batch_indexes = np.arange(batch_size * batch, batch_size * (batch + 1))             
                current_batch_ratings = train[mixed_order[batch_indexes], :]
                sample_test = test[np.random.choice(num_tests, batch_size), :]
                self.update(current_batch_ratings)
                train_loss = self.loss(current_batch_ratings)
                test_loss  = self.loss(sample_test)
                avg_train_loss.append(train_loss)
                avg_test_loss.append(test_loss)
            
            avg_train_loss = np.mean(avg_train_loss) / float(batch_size) 
            avg_test_loss  = np.mean(avg_test_loss) / float(batch_size)
            print(f'{arrow.now()} Epoch № {self.e}', file=sys.stderr)
            print(f'{arrow.now()} Training loss:\t{avg_train_loss}', file=sys.stderr)
            print(f'{arrow.now()} Testing loss:\t{avg_test_loss}', file=sys.stderr)

In [27]:
df_review = pd.read_csv("review_data_state.csv")
df_review["text"].fillna('', inplace=True)
print(df_review.head())
df_review = df_review[['user_id', 'business_id', 'stars', 'text']]
dict_users = {key: val for val, key in enumerate(df_review['user_id'].unique())}
inv_dict_users = {val: key for key, val in dict_users.items()}
dict_items = {key: val for val, key in enumerate(df_review['business_id'].unique())}
inv_dict_items = {val: key for key, val in dict_items.items()}
    
df_review['user_id'] = df_review['user_id'].swifter.apply(lambda x: dict_users[x])
df_review['business_id'] = df_review['business_id'].swifter.apply(lambda x: dict_items[x])
    
train_ratings_df, test_ratings_df = train_test_split(df_review, test_size = 0.2)

print(f'Size of training data: {train_ratings_df.shape}')
print(f'Size of test data: {test_ratings_df.shape}')

                review_id                 user_id             business_id  \
0  tanL1f9UuVPE2veDDHAL-w  h_kjYBpubAmYVxvNaID7Lw  i9BDFBYcl_PGqrLbQUdMvg   
1  zDlr3bWWZRUJ-0JGBXZwAw  kP92EoMqvHJVnoSaPXtShg  5P3b82EqZqR7lCqAYj-zaQ   
2  fVyVgYYbFsxV5XP1a-YwcQ  Kkxi0nhaXi16TgFd7olCFQ  KRNrssbpE1AKo-48BIpdLw   
3  s3RbFgeaWisnyIBZanlpIw  4gCXQdAi0Rbt4QTLc7Xz9w  8IhF8Vvnr3Nz-b3GR11M6A   
4  k9vlSSUStwY2DcjM8Rinnw  7C8RwnXqNhj6mYxowI53sA  Of6xu3pY3eHe2yhiyz2dvg   

   stars                                               text  
0    4.0  excellent sub reasonable price huge variety pi...  
1    4.0  great quick service despite fact every table f...  
2    4.0  place really good service fast efficient peopl...  
3    4.0  great staff wonderful service wine list top no...  
4    4.0  literally thing anything order right gave enou...  



Size of training data: (54682, 4)
Size of test data: (13671, 4)


In [28]:
def get_train_whole_text(df, name_id):
    return df[[name_id,'text']].groupby(name_id).agg({'text': ' '.join})
    
    
user_id_review_df = pd.DataFrame(index=inv_dict_users.keys(), columns=["text"])
user_id_review_df.index.name = 'user_id'
business_id_review_df= pd.DataFrame(index=inv_dict_items.keys(), columns=["text"])
business_id_review_df.index.name = 'business_id'

user_id_review_df['text'] =  get_train_whole_text(train_ratings_df, 'user_id')["text"]
business_id_review_df['text'] =  get_train_whole_text(train_ratings_df, 'business_id')["text"]
user_id_review_df["text"].fillna('', inplace=True)
business_id_review_df["text"].fillna('', inplace=True)

In [31]:
business_id_vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, max_features=5000)
business_id_vectors = business_id_vectorizer.fit_transform(business_id_review_df['text'])

user_id_vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, max_features=5000)
user_id_vectors = user_id_vectorizer.fit_transform(user_id_review_df['text'])

U = pd.DataFrame(user_id_vectors.toarray(), index=user_id_review_df.index, columns=user_id_vectorizer.get_feature_names())
U.sort_index(inplace=True)
I = pd.DataFrame(business_id_vectors.toarray(), index=business_id_review_df.index, columns=business_id_vectorizer.get_feature_names())
I.sort_index(inplace=True)

matrixF = MatrixFactorization(U = U.values, I = I.values, learning_rate=0.07, lamda=0.1, num_epoches=8, num_batches=1000)

In [32]:
R = pd.pivot_table(df_review, values='stars', index=['user_id'], columns=['business_id'])
R.head()

business_id,0,1,2,3,4,5,6,7,8,9,...,487,488,489,490,491,492,493,494,495,496
user_id,,,,,,,,,,,,,,,,,,,,,
0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,4.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
matrixF.fit(train_ratings_df.values[:, 0:3], test_ratings_df.values[:, 0:3])

2021-09-08T01:10:48.966577-07:00 Epoch № 1
2021-09-08T01:10:48.978577-07:00 Training loss:	2.708472173452243
2021-09-08T01:10:48.980579-07:00 Testing loss:	11.042344948179174
2021-09-08T01:10:52.851577-07:00 Epoch № 2
2021-09-08T01:10:52.852579-07:00 Training loss:	0.5473154930797792
2021-09-08T01:10:52.852579-07:00 Testing loss:	9.208049576893792
2021-09-08T01:10:56.780587-07:00 Epoch № 3
2021-09-08T01:10:56.781587-07:00 Training loss:	0.5037503769448762
2021-09-08T01:10:56.782587-07:00 Testing loss:	9.091342024558932
2021-09-08T01:11:00.707610-07:00 Epoch № 4
2021-09-08T01:11:00.708609-07:00 Training loss:	0.49377015857254897
2021-09-08T01:11:00.709610-07:00 Testing loss:	8.90368169561439
2021-09-08T01:11:04.711850-07:00 Epoch № 5
2021-09-08T01:11:04.712850-07:00 Training loss:	0.48772085793321657
2021-09-08T01:11:04.713850-07:00 Testing loss:	9.05619238613045
2021-09-08T01:11:08.776963-07:00 Epoch № 6
2021-09-08T01:11:08.777964-07:00 Training loss:	0.4835910229819247
2021-09-08T01:1

In [35]:
U = pd.DataFrame(matrixF.U, index=user_id_review_df.index, columns=user_id_vectorizer.get_feature_names())
U.sort_index(inplace=True)
I = pd.DataFrame(matrixF.I, index=business_id_review_df.index, columns=business_id_vectorizer.get_feature_names())
I.sort_index(inplace=True)

In [36]:
if __name__ == '__main__':
    R = np.dot(matrixF.U, matrixF.I.T)
    with Pool(processes = 10) as pool:
        R = np.array(pool.map(correct_numbers, R))

In [37]:
R = pd.DataFrame(R, index=U.index, columns=I.index)

In [38]:
print(f"Predicted ratings:\n")
R.head()

Predicted ratings:



business_id,0,1,2,3,4,5,6,7,8,9,...,487,488,489,490,491,492,493,494,495,496
user_id,,,,,,,,,,,,,,,,,,,,,
0,3.0,4.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0
1,3.0,4.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,3.0,...,5.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0
2,3.0,4.0,2.0,3.0,1.0,3.0,3.0,3.0,2.0,3.0,...,3.0,4.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0
3,2.0,3.0,2.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
I.iloc[train_ratings_df.iat[0, 1]].sort_values(ascending=False).head(10)

chicken      1.047385
fried        0.636507
good         0.418436
place        0.397927
food         0.377281
gluten       0.369265
really       0.366488
side         0.335338
delicious    0.333114
great        0.327097
Name: 169, dtype: float64

In [40]:
U.iloc[train_ratings_df.iat[0, 0]].sort_values(ascending=False).head(10)

chinese       0.815604
food          0.433050
dish          0.290703
spice         0.277788
five          0.253621
restaurant    0.244249
order         0.241854
authentic     0.228221
sesame        0.213176
spoken        0.209295
Name: 14187, dtype: float64

In [41]:
data = {'U': U, 'I': I, 'V': user_id_vectorizer}

with open('recommendation_model.pkl', 'wb') as f:
     pickle.dump(data, f)

In [42]:
R.to_csv('predicted_ratings.csv', sep='\t')

In [43]:
with open('recommendation_model.pkl', 'rb') as f:
     model = pickle.load(f)
print(model)

{'U':               ability      able       abo  absolute  absolutely    absurd  \
user_id                                                                     
0        1.352703e-04  0.018624  0.000381  0.007396    0.196555  0.001651   
1       -3.296113e-03  0.019340  0.022321 -0.000672    0.032348  0.000403   
2        1.694067e-03  0.009752  0.000251  0.003612    0.021944  0.003076   
3        2.540132e-03  0.023896 -0.003115 -0.000056    0.019124  0.001276   
4        0.000000e+00  0.000000  0.000000  0.000000    0.000000  0.000000   
5        6.241953e-05  0.008994 -0.000328  0.002478    0.019066 -0.000051   
6        3.645860e-03  0.040025  0.000380  0.005351    0.060570 -0.000566   
7       -3.451459e-03  0.009495  0.003776 -0.017376    0.021638 -0.002738   
8        2.660729e-03  0.009377  0.004886  0.018929    0.048587 -0.000350   
9       -3.610939e-04  0.003180  0.008109 -0.001799    0.037809  0.003941   
10       2.075616e-02  0.007557  0.001352  0.007168    0.080063 -0.000

        use_idf=True, vocabulary=None)}


In [44]:
I = model["I"]
U = model["U"]
user_id_vectorizer = model["V"] 

In [45]:
R = pd.read_csv('predicted_ratings.csv', sep='\t')
R.head()

,user_id,0,1,2,3,4,5,6,7,8,...,487,488,489,490,491,492,493,494,495,496
0,0,3.0,4.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0
1,1,3.0,4.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,...,5.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0
2,2,3.0,4.0,2.0,3.0,1.0,3.0,3.0,3.0,2.0,...,3.0,4.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0
3,3,2.0,3.0,2.0,4.0,1.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
words = input("What place are you looking for? ")

query = pd.DataFrame([text_preprocess(words)], columns=['text']) 
query_vector = user_id_vectorizer.transform(query['text']) 
query = pd.DataFrame(query_vector.toarray(), index=query.index, columns=user_id_vectorizer.get_feature_names())

predictRating=pd.DataFrame(np.dot(query.loc[0],I.T), index=I.index, columns=['rating'])
topRecommendations=pd.DataFrame.sort_values(predictRating,['rating'],ascending=[0])[:5]

number = 0
for i in topRecommendations.index:
    number+=1
    print(f"\n№ {number}")
    print(f"Name: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['name'].iloc[0]}")
    print(f"City, state: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['city'].iloc[0]}, {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['state'].iloc[0]}")
    print(f"Category: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['categories'].iloc[0]}")
    print(f"Average star: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['stars'].iloc[0]}")
    print(f"Review count: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['review_count'].iloc[0]}")

What place are you looking for? Chinese food

№ 1
Name: Five Spice Asian Cuisine
City, state: Boulder, CO
Category: Restaurants, Vegetarian, Thai, Chinese
Average star: 2.5
Review count: 116

№ 2
Name: Haoway Chinese Cafe
City, state: Boulder, CO
Category: Chinese, Restaurants
Average star: 3.5
Review count: 85

№ 3
Name: Golden Sun
City, state: Boulder, CO
Category: Restaurants, Chinese
Average star: 2.5
Review count: 138

№ 4
Name: Spice China
City, state: Louisville, CO
Category: Restaurants, Chinese
Average star: 3.0
Review count: 187

№ 5
Name: China Gourmet
City, state: Boulder, CO
Category: Restaurants, Chinese
Average star: 4.0
Review count: 129
